In [134]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as sf
from pyspark.sql import Window

spark = SparkSession.builder.appName('Jupyter')\
    .config(map={
        'spark.sql.autoBroadcastJoinThreshold': -1
    })\
    .getOrCreate()

In [135]:
%%sql
SHOW TABLES FROM bootcamp

namespace,tableName,isTemporary
bootcamp,events,False
bootcamp,events_sorted,False
bootcamp,events_unsorted,False
bootcamp,maps,False
bootcamp,match_details,False
bootcamp,match_details_bucketed,False
bootcamp,match_player_models,False
bootcamp,matches,False
bootcamp,matches_bucketed,False
bootcamp,medals,False


In [136]:
%%sql
SHOW NAMESPACES

namespace
bootcamp


In [137]:
matches = spark.read.options(
    header = True,
    inferSchema = True
).csv('/home/iceberg/data/matches.csv')

match_details = spark.read.options(
    header = True,
    inferSchema = True
).csv('/home/iceberg/data/match_details.csv')

medals_matches_players = spark.read.options(
    header = True,
    inferSchema = True
).csv('/home/iceberg/data/medals_matches_players.csv')

medals = spark.read.options(
    header = True,
    inferSchema = True
).csv('/home/iceberg/data/medals.csv')

maps = spark.read.options(
    header = True,
    inferSchema = True
).csv('/home/iceberg/data/maps.csv')

In [138]:
matches.describe().show()
match_details.describe().show()
medals_matches_players.describe().show()
medals.describe().show()
maps.describe().show()

+-------+--------------------+--------------------+--------------------+--------------------+--------------+---------+--------------------+
|summary|            match_id|               mapid|         playlist_id|     game_variant_id|match_duration|game_mode|      map_variant_id|
+-------+--------------------+--------------------+--------------------+--------------------+--------------+---------+--------------------+
|  count|               24025|               24025|               24025|               24025|             0|        0|                9516|
|   mean|                NULL|                NULL|                NULL|                NULL|          NULL|     NULL|                NULL|
| stddev|                NULL|                NULL|                NULL|                NULL|          NULL|     NULL|                NULL|
|    min|0000e3cf-727c-491...|5e130537-2275-40b...|0504ca3c-de41-48f...|1571fdac-e0b4-4eb...|          NULL|     NULL|00e24014-e0c7-44b...|
|    max|fffc65f4-bc

+-------+--------------------+---------------+---------------------+-----------------+------------------+------------------+------------------+------------------------+-----------------+---------------------------------+------------------+------------------+-----------------------+-----------------+--------------------------------+------------------+-------------------+-------------------+------------------+----------------------+--------------------------+-------------------------+------------------------+-------------------------+---------------------------+-------------------------------+--------------------------------+---------------------------+--------------------------------+-------------------------------+-------------------+--------------------+--------------------------+-------------------+------------------+
|summary|            match_id|player_gamertag|previous_spartan_rank|     spartan_rank| previous_total_xp|          total_xp| previous_csr_tier|previous_csr_designation|

+-------+--------------------+---------------+--------------------+------------------+
|summary|            match_id|player_gamertag|            medal_id|             count|
+-------+--------------------+---------------+--------------------+------------------+
|  count|              755229|         755229|              755229|            755229|
|   mean|                NULL|           NULL|2.2952806324313316E9|2.0661891955949785|
| stddev|                NULL|           NULL|1.0615671192174464E9| 2.383679012860514|
|    min|0000e3cf-727c-491...|  A 0 N Eclipse|            35545941|                 1|
|    max|fffc65f4-bc88-400...|    zzzgameszzz|          4252521258|                58|
+-------+--------------------+---------------+--------------------+------------------+

+-------+--------------------+--------------------+-----------------+------------------+------------------+-------------------+------------------+------------------+-----------------+--------------------+------------

In [139]:
matches.schema.fields

[StructField('match_id', StringType(), True),
 StructField('mapid', StringType(), True),
 StructField('is_team_game', BooleanType(), True),
 StructField('playlist_id', StringType(), True),
 StructField('game_variant_id', StringType(), True),
 StructField('is_match_over', BooleanType(), True),
 StructField('completion_date', TimestampType(), True),
 StructField('match_duration', StringType(), True),
 StructField('game_mode', StringType(), True),
 StructField('map_variant_id', StringType(), True)]

In [140]:
(
    matches
        .writeTo('bootcamp.matches')
        .tableProperty('write.bucket-by-columns', 'match_id, playlist_id')
        .tableProperty('write.bucket-by-number', '4')
        .partitionedBy(
            sf.years('completion_date'),
            'mapid'
        )
        .createOrReplace()
)

In [141]:
match_details.schema.fields

[StructField('match_id', StringType(), True),
 StructField('player_gamertag', StringType(), True),
 StructField('previous_spartan_rank', IntegerType(), True),
 StructField('spartan_rank', IntegerType(), True),
 StructField('previous_total_xp', IntegerType(), True),
 StructField('total_xp', IntegerType(), True),
 StructField('previous_csr_tier', IntegerType(), True),
 StructField('previous_csr_designation', IntegerType(), True),
 StructField('previous_csr', IntegerType(), True),
 StructField('previous_csr_percent_to_next_tier', IntegerType(), True),
 StructField('previous_csr_rank', IntegerType(), True),
 StructField('current_csr_tier', IntegerType(), True),
 StructField('current_csr_designation', IntegerType(), True),
 StructField('current_csr', IntegerType(), True),
 StructField('current_csr_percent_to_next_tier', IntegerType(), True),
 StructField('current_csr_rank', IntegerType(), True),
 StructField('player_rank_on_team', IntegerType(), True),
 StructField('player_finished', Boolea

In [142]:
(
    match_details
        .writeTo('bootcamp.match_details')
        .tableProperty('write.bucket-by-columns', 'match_id')
        .tableProperty('write.bucket-by-number', '4')
        .createOrReplace()
)

25/08/08 17:26:25 WARN S3FileIO: Unclosed S3FileIO instance created by:+ 7) / 7]
	org.apache.iceberg.aws.s3.S3FileIO.initialize(S3FileIO.java:371)
	org.apache.iceberg.CatalogUtil.loadFileIO(CatalogUtil.java:369)
	org.apache.iceberg.rest.RESTSessionCatalog.newFileIO(RESTSessionCatalog.java:1042)
	org.apache.iceberg.rest.RESTSessionCatalog.initialize(RESTSessionCatalog.java:323)
	org.apache.iceberg.rest.RESTCatalog.initialize(RESTCatalog.java:78)
	org.apache.iceberg.CatalogUtil.loadCatalog(CatalogUtil.java:274)
	org.apache.iceberg.CatalogUtil.buildIcebergCatalog(CatalogUtil.java:328)
	org.apache.iceberg.spark.SparkCatalog.buildIcebergCatalog(SparkCatalog.java:153)
	org.apache.iceberg.spark.SparkCatalog.initialize(SparkCatalog.java:752)
	org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:65)
	org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$catalog$1(CatalogManager.scala:54)
	scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	org.apache.spa

In [143]:
medals_matches_players.schema.fields

[StructField('match_id', StringType(), True),
 StructField('player_gamertag', StringType(), True),
 StructField('medal_id', LongType(), True),
 StructField('count', IntegerType(), True)]

In [144]:
(
    medals_matches_players
        .writeTo('bootcamp.medals_matches_players')
        .tableProperty('write.bucket-by-columns', 'match_id, player_gamertag, medal_id')
        .tableProperty('write.bucket-by-number', '4')
        .createOrReplace()
)

In [145]:
medals.schema.fields

[StructField('medal_id', LongType(), True),
 StructField('sprite_uri', StringType(), True),
 StructField('sprite_left', IntegerType(), True),
 StructField('sprite_top', IntegerType(), True),
 StructField('sprite_sheet_width', IntegerType(), True),
 StructField('sprite_sheet_height', IntegerType(), True),
 StructField('sprite_width', IntegerType(), True),
 StructField('sprite_height', IntegerType(), True),
 StructField('classification', StringType(), True),
 StructField('description', StringType(), True),
 StructField('name', StringType(), True),
 StructField('difficulty', IntegerType(), True)]

In [146]:
(
    medals
        .writeTo('bootcamp.medals')
        .tableProperty('write.bucket-by-columns', 'medal_id')
        .tableProperty('write.bucket-by-number', '4')
        .createOrReplace()
)

In [147]:
maps.schema.fields

[StructField('mapid', StringType(), True),
 StructField('name', StringType(), True),
 StructField('description', StringType(), True)]

In [148]:
(
    maps
        .writeTo('bootcamp.maps')
        .tableProperty('write.bucket-by-columns', 'mapid')
        .tableProperty('write.bucket-by-number', '4')
        .createOrReplace()
)

In [149]:
match_player_medals = spark.table('bootcamp.match_details').alias('md').join(
    spark.table('bootcamp.matches').alias('m'),
    'match_id'
).join(
    spark.table('bootcamp.medals_matches_players').alias('mmp'),
    ['match_id', 'player_gamertag']
).cache()

match_player_medals.show(truncate=False)

+------------------------------------+---------------+---------------------+------------+-----------------+--------+-----------------+------------------------+------------+---------------------------------+-----------------+----------------+-----------------------+-----------+--------------------------------+----------------+-------------------+---------------+-------------------+------------------+----------------------+--------------------------+-------------------------+------------------------+-------------------------+---------------------------+-------------------------------+--------------------------------+---------------------------+--------------------------------+-------------------------------+-------------------+--------------------+--------------------------+-------+-------+------------------------------------+------------+------------------------------------+------------------------------------+-------------+-------------------+--------------+---------+------------------

In [150]:
match_player_medals.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- InMemoryTableScan [match_id#81154, player_gamertag#81155, previous_spartan_rank#81156, spartan_rank#81157, previous_total_xp#81158, total_xp#81159, previous_csr_tier#81160, previous_csr_designation#81161, previous_csr#81162, previous_csr_percent_to_next_tier#81163, previous_csr_rank#81164, current_csr_tier#81165, current_csr_designation#81166, current_csr#81167, current_csr_percent_to_next_tier#81168, current_csr_rank#81169, player_rank_on_team#81170, player_finished#81171, player_average_life#81172, player_total_kills#81173, player_total_headshots#81174, player_total_weapon_damage#81175, player_total_shots_landed#81176, player_total_melee_kills#81177, ... 23 more fields]
      +- InMemoryRelation [match_id#81154, player_gamertag#81155, previous_spartan_rank#81156, spartan_rank#81157, previous_total_xp#81158, total_xp#81159, previous_csr_tier#81160, previous_csr_designation#81161, previous_csr#81162, previous_csr_percent_to_nex

In [151]:
match_players_stats = match_player_medals.groupBy(
    'match_id',
    'player_gamertag'
).agg(
    sf.max('mapid').alias('map_id'),
    sf.max('playlist_id').alias('playlist_id'),
    sf.max('player_total_kills').alias('kills'),
    sf.max('player_total_deaths').alias('deaths'),
    sf.max('player_total_assists').alias('assists')
).cache() 

match_players_stats.show(truncate=False)

+------------------------------------+---------------+------------------------------------+------------------------------------+-----+------+-------+
|match_id                            |player_gamertag|map_id                              |playlist_id                         |kills|deaths|assists|
+------------------------------------+---------------+------------------------------------+------------------------------------+-----+------+-------+
|00595f68-ff27-48fb-99f5-5d55afcdefee|Indigo Ferocity|cd844200-f206-11e4-9393-24be05e24f7e|f27a65eb-2d11-4965-aa9c-daa088fa5c9c|13   |9     |3      |
|00639af5-d1a8-438c-a746-0299a36dc4db|Alpha Atra     |cc040aa1-f206-11e4-a3e0-24be05e24f7e|2323b76a-db98-4e03-aa37-e171cfbdd1a4|7    |13    |2      |
|00925e62-5074-4f7b-a377-09b916ea4dcf|B1aZinHeadZ    |caacb800-f206-11e4-81ab-24be05e24f7e|f27a65eb-2d11-4965-aa9c-daa088fa5c9c|10   |14    |3      |
|00a24cd5-4e90-475a-ad39-bb0170229d3a|II JAMADDICT II|c7edbf0f-f206-11e4-aa52-24be05e24f7e|f72e0ef0-

In [165]:
(
    match_players_stats
    .groupBy('player_gamertag')
    .agg(
        sf.avg('kills').alias('avg_kills')
    )
    .sort(sf.desc('avg_kills'))
    .take(1)
)

[Row(player_gamertag='gimpinator14', avg_kills=109.0)]

In [167]:
(
    match_players_stats
    .groupBy('playlist_id')
    .agg(
        sf.count('playlist_id').alias('count')
    )
    .sort(sf.desc('count'))
    .take(1)
)

[Row(playlist_id='f72e0ef0-7c4a-4307-af78-8e38dac3fdba', count=55400)]

25/08/08 19:57:36 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2250555 ms exceeds timeout 120000 ms
25/08/08 19:57:36 WARN SparkContext: Killing executors is not supported by current scheduler.
25/08/08 19:57:40 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 

In [154]:
(
    match_players_stats.alias('m')
    .join(
        spark.table('bootcamp.maps').alias('mp'),
        sf.col('m.map_id') == sf.col('mp.mapid')
    )
    .groupBy('map_id')
    .agg(
        sf.max('name'),
        sf.count('map_id').alias('count')
    )
    .sort(sf.desc('count'))
    .take(1)
)

[Row(map_id='c7edbf0f-f206-11e4-aa52-24be05e24f7e', max(name)='Breakout Arena', count=50741)]

In [155]:
(
    match_player_medals
    .join(
        sf.broadcast(spark.table('bootcamp.medals')),
        'medal_id'
    )
    .alias('d')
    .join(
        sf.broadcast(spark.table('bootcamp.maps')).alias('m'),
        'mapid'
    )
    .where('d.name = "Killing Spree"')
    .groupBy('mapid')
    .agg(
        sf.max('m.name').alias('map_name'),
        sf.max('d.name').alias('medal_name'),
        sf.sum('count').alias('count')
    )
    .sort(sf.desc('count'))
    .take(1)
)

[Row(mapid='c7edbf0f-f206-11e4-aa52-24be05e24f7e', map_name='Breakout Arena', medal_name='Killing Spree', count=6738)]

In [156]:
(
    match_player_medals
    .writeTo('bootcamp.match_player_models')
    .tableProperty('write.bucket-by-columns', 'match_id')
    .tableProperty('write.bucket-by-number', '16')
    .createOrReplace()
)

In [157]:
spark.table('bootcamp.match_player_models.files').select(sf.sum('file_size_in_bytes')).show()

+-----------------------+
|sum(file_size_in_bytes)|
+-----------------------+
|               24473827|
+-----------------------+



In [158]:
(
    match_player_medals
    .sort('mapid')
    .writeTo('bootcamp.match_player_models')
    .tableProperty('write.bucket-by-columns', 'match_id')
    .tableProperty('write.bucket-by-number', '16')
    .createOrReplace()
)

In [159]:
spark.table('bootcamp.match_player_models.files').select(sf.sum('file_size_in_bytes')).show()

+-----------------------+
|sum(file_size_in_bytes)|
+-----------------------+
|               17126996|
+-----------------------+



In [160]:
(
    match_player_medals
    .sort('playlist_id')
    .writeTo('bootcamp.match_player_models')
    .tableProperty('write.bucket-by-columns', 'match_id')
    .tableProperty('write.bucket-by-number', '16')
    .createOrReplace()
)

In [161]:
spark.table('bootcamp.match_player_models.files').select(sf.sum('file_size_in_bytes')).show()

+-----------------------+
|sum(file_size_in_bytes)|
+-----------------------+
|               16428831|
+-----------------------+



In [162]:
(
    match_player_medals
    .sort('mapid', 'playlist_id')
    .writeTo('bootcamp.match_player_models')
    .tableProperty('write.bucket-by-columns', 'match_id')
    .tableProperty('write.bucket-by-number', '16')
    .createOrReplace()
)

In [163]:
spark.table('bootcamp.match_player_models.files').select(sf.sum('file_size_in_bytes')).show()

+-----------------------+
|sum(file_size_in_bytes)|
+-----------------------+
|               16749987|
+-----------------------+

